In [10]:
def getData(inputFiles):
    cols = []
    scalar = []
    scalar.append('evid')
    scalar.append('xsec')
    scalar.append('ntrials')    
    scalar.append('x')
    scalar.append('y')
    scalar.append('Q2')
    scalar.append('W2')
    scalar.append('pt_jet')
    scalar.append('eta_jet')
    scalar.append('phi_jet')
    scalar.append('pt_quark')
    scalar.append('eta_quark')
    scalar.append('phi_quark')
    scalar.append('pt_photon')
    scalar.append('phi_photon')
    scalar.append('eta_electron')
    scalar.append('pt_electron')
    scalar.append('phi_electron')
    lists = scalar
    dataframes = []
    for inputFilename in inputFiles:
        start = time.time()
        df = read_root(inputFilename, columns=lists,flatten=cols)
        dataframes.append(df)
        end = time.time()
        print '\n', 'Processed', inputFilename, 'in',  '%2.1f'%(end-start), 'seconds'
    return pd.concat([x for x in dataframes])

In [1]:
def polar(Q2Index, xIndex, r_low, theta_low, part_low, r_high, theta_high, color):
    if xIndex > 4 and Q2Index > 0:
        radii = radii_large
        r_labels = r_labels_large
        nr = nr_large
    else:    
        radii = radii_small
        r_labels = r_labels_small
        nr = nr_small
    r_edges = np.linspace(0, nr, nr + 1)
    theta_edges = np.linspace(0, 2*np.pi, ntheta + 1)
    H_e = plt.hist2d(r_high, theta_high, [r_edges, theta_edges])[0]
    H_q = plt.hist2d(r_low, theta_low, [r_edges, theta_edges])[0]
    plt.cla()
    
    Theta, R = np.meshgrid(theta_edges, r_edges)
    im_e = plt.pcolormesh(Theta, R, H_e, cmap='Blues', norm=LogNorm())
    im_q = plt.pcolormesh(Theta, R, H_q, cmap=color, norm=LogNorm())
    plt.rgrids(radii, labels=r_labels, fontsize=20)
    plt.thetagrids(angles, labels=eta_labels, fontsize=20)
    plt.grid(color='grey')
    cbar_e = plt.colorbar(im_e, shrink=0.7, pad=0.01, cax=plt.axes([0.85, 0.55, 0.03, 0.3]))
    cbar_e.ax.tick_params(labelsize=20)
    cbar_q = plt.colorbar(im_q, shrink=0.7, pad=0.01, cax=plt.axes([0.85, 0.2, 0.03, 0.3]))
    cbar_q.ax.tick_params(labelsize=20)

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


In [1]:
def getDataPart(inputFiles):
    cols = []
    scalar = []
    scalar.append('evid')
    scalar.append('xsec')
    scalar.append('ntrials')    
    scalar.append('x')
    scalar.append('y')
    scalar.append('Q2')
    scalar.append('W2')
    scalar.append('id_quark')
    scalar.append('pt_quark')
    scalar.append('eta_quark')
    scalar.append('eta_electron')
    cols.append('jetid')
    cols.append('pt_jet')
    cols.append('eta_jet')
    cols.append('id_lead_part')
    cols.append('id_part')
    cols.append('pT_part')
    cols.append('eta_part')
    cols.append('theta_part')
    lists = scalar + cols
    dataframes = []
    for inputFilename in inputFiles:
        start = time.time()
        df = read_root(inputFilename, columns=lists,flatten=cols)
        dataframes.append(df)
        end = time.time()
        print '\n', 'Processed', inputFilename, 'in',  '%2.1f'%(end-start), 'seconds'
    return pd.concat([x for x in dataframes])

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


In [5]:
def getParticle(array):
    particle_array = []
    for array_element in array:
        if (array_element == 2212):
            particle_array.append('$p$/$\overline{p}$')
        elif (array_element == 2112):
            particle_array.append('$n$/$\overline{n}$')
        elif (array_element == 321):
            particle_array.append('$K^{+}$/$K^{-}$')
        elif (array_element == 211):
            particle_array.append('$\pi^{+}$/$\pi^{-}$')
        elif (array_element == 130):
            particle_array.append('$K^0_L$')
        elif (array_element == 22):
            particle_array.append('$\gamma$')
        elif (array_element == 16):
            particle_array.append(r'$\nu_T$/$\overline{\nu}_T$')
        elif (array_element == 14):
            particle_array.append(r'$\nu_{\mu}$/$\overline{\nu}_{\mu}$')
        elif (array_element == 13):
            particle_array.append('$\mu^{-}$/$\mu^{+}$')
        elif (array_element == 12):
            particle_array.append(r'$\nu_e$/$\overline{\nu}_e$')
        elif (array_element == 11):
            particle_array.append('$e^{-}$/$e^{+}$')
        else:
            particle_array.append(str(array_element))
    return particle_array

In [3]:
def getQuark(array):
    particle_array = []
    for array_element in array:
        if (array_element == -5 or array_element == 5):
            particle_array.append('$b$/$\overline{b}$')
        elif (array_element == -4 or array_element == 4):
            particle_array.append('$c$/$\overline{c}$')
        elif (array_element == -3 or array_element == 3):
            particle_array.append('$s$/$\overline{s}$')
        elif (array_element == -2):
            particle_array.append('$\overline{u}$')
        elif (array_element == -1):
            particle_array.append('$\overline{d}$')
        elif (array_element == 1):
            particle_array.append('$d$')
        elif (array_element == 2):
            particle_array.append('$u$')
        else:
            particle_array.append(str(array_element))
    return particle_array

In [4]:
def getIdAndCounts(df, var, other=True):
    b = df.groupby(var).count()
    id_table = np.array([b.index,b['Q2']])
    if other == False:
        return id_table
    else:
        other_count_tot = 0
        other_id_array = []
        other_count_array = []
        remove_array = []
        for i in range(0, len(id_table[0])):
            if float(id_table[1][i])/sum(id_table[1]) < 0.01:
                other_id_array.append(id_table[0][i])
                other_count_array.append(id_table[1][i])
                other_count_tot += id_table[1][i]
                remove_array.append(i)
        id_table = np.delete(id_table, remove_array, 1)
        other_id_table = np.array([other_id_array, other_count_array])
        return id_table, other_id_table, other_count_tot

In [4]:
def getRanges(edges):
    return zip(edges[:-1], edges[1:])

In [2]:
def applyCut(inputDataframe, cut, text=None):
    dataframe = inputDataframe
    nbeforecut = dataframe.shape[0]
    cutDataframe = dataframe.query(cut)
    if text:
        print text, cutDataframe.shape[0], ' fraction kept: %2.1f'%(100.0*float(cutDataframe.shape[0])/nbeforecut)
    return cutDataframe

In [12]:
def applyCuts(df, xedges, Q2edges):
    xranges = getRanges(xedges)
    Q2ranges = getRanges(Q2edges)
    ds = [['' for j in range(0,len(xranges))] for i in range(0,len(Q2ranges))]
    for i in range(0,len(Q2ranges)):
        Q2low, Q2high = Q2ranges[i]
        print '\n', str(Q2low) +' < Q2 < '+ str(Q2high) +' GeV2', '\n'
        ds_Q2 = applyCut(df, 'Q2 > ' + str(Q2low) + 'and Q2 < ' + str(Q2high), str(Q2low) + ' < Q2 < ' + str(Q2high))
        for j in range(0,len(xranges)):
            xlow, xhigh = xranges[j]
            ds_Q2_x = applyCut(ds_Q2, 'x > ' + str(xlow) + 'and x < ' + str(xhigh), str(xlow) + ' < x < ' + str(xhigh))
            ds[i][j] = ds_Q2_x
    return ds

In [3]:
def partitionEtaPt(df, etaedges, ptranges):
    etaranges = getRanges(etaedges)
    ptranges = getRanges(ptranges)
    ds = [['' for j in range(0,len(ptranges))] for i in range(0,len(etaranges))]
    for i in range(0,len(etaranges)):
        etalow, etahigh = etaranges[i]
        print '\n', str(etalow) +' < eta_jet < '+ str(etahigh), '\n'
        ds_eta = applyCut(df, 'eta_jet> ' + str(etalow) + 'and eta_jet < ' + str(etahigh), str(etalow) + ' < eta_jet < ' + str(etahigh))
        for j in range(0,len(ptranges)):
            ptlow, pthigh = ptranges[j]
            ds_eta_pt = applyCut(ds_eta, 'pt_jet > ' + str(ptlow) + 'and pt_jet < ' + str(pthigh), str(ptlow) + ' < pt_jet < ' + str(pthigh))
            ds[i][j] = ds_eta_pt
    return ds

In [13]:
def getNumber(ds):
    matrix = [['' for j in range(0,len(ptedges)-1)] for i in range(0,len(etaedges)-1)]
    for i in range(0,len(etaedges)-1):
        for j in range(0,len(ptedges)-1):
            matrix[i][j] = ds[i][j].shape[0]
    return matrix

In [11]:
def partitionEta(df, etaedges):
    etaranges = getRanges(etaedges)
    ds = ['' for i in range(0,len(etaranges))]
    for i in range(0,len(etaranges)):
        etalow, etahigh = etaranges[i]
        print '\n', str(etalow) +' < eta_jet < '+ str(etahigh), '\n'
        ds[i] = applyCut(df, 'eta_jet> ' + str(etalow) + 'and eta_jet < ' + str(etahigh), str(etalow) + ' < eta_jet < ' + str(etahigh))
    return ds

In [5]:
def keepOneJet(inputDataframe):    
    dataframe = inputDataframe
    nbeforecut = dataframe.shape[0]
    cutDataframe = dataframe.loc[~dataframe['evid'].duplicated(keep='first')]
    print '\n','Remove extra jets',cutDataframe.shape[0],' fraction kept: %2.1f'%(100.0*float(cutDataframe.shape[0])/nbeforecut)
    return cutDataframe

In [4]:
def keepOnePart(inputDataframe):    
    dataframe = inputDataframe
    nbeforecut = dataframe.shape[0]
    cutDataframe = dataframe.loc[~dataframe['jetid'].duplicated(keep='first')]
    print '\n','Remove extra part',cutDataframe.shape[0],' fraction kept: %2.1f'%(100.0*float(cutDataframe.shape[0])/nbeforecut)
    return cutDataframe

In [6]:
# Doesn't work
def fixPhi(df_o, particle, groomed=False):
    df = df_o
    if groomed == True:
        string = 'phi_jet_g'
    else:
        string = 'phi_jet'
    a = (df.loc[df['phi_' + particle] < 0]).index # a list of row indices (jet id) with negative phi_photon values
    for i in range(0,len(a)):
        df.at[a[i],'phi_' + particle] = df.at[a[i],'phi_' + particle] + 6.28318530718
    print 'Fixed ' + particle + ' phi range'
    eq = 'delta_phi_' + particle + ' = phi_' + particle + ' - ' + string
    df.eval(eq, inplace=True)
    b = (df.loc[df['delta_phi_' + particle] > np.pi]).index
    for i in range(0,len(b)):
        df.at[b[i],'delta_phi_' + particle] = -6.28318530718 + df.at[b[i],'delta_phi_' + particle]
    c = (df.loc[df['delta_phi_' + particle] < (-1) * np.pi]).index
    for i in range(0,len(c)):
        df.at[c[i],'delta_phi_' + particle] = 6.28318530718 + df.at[c[i],'delta_phi_' + particle]
    df.eval('delta_phi_' + particle + '_abs = abs(delta_phi_' + particle + ')', inplace=True)    
    print 'Fixed ' + particle + ' delta phi range'
    return df

In [6]:
def fixPhi(df, particle, groomed=False):
    if groomed == True:
        string = 'phi_jet_g'
    else:
        string = 'phi_jet'
    a = (df.loc[df['phi_' + particle] < 0]).index # a list of row indices (jet id) with negative phi_photon values
    for i in range(0,len(a)):
        df.at[a[i],'phi_' + particle] = df.at[a[i],'phi_' + particle] + 6.28318530718
    print 'Fixed ' + particle + ' phi range'
    df_new = df.eval('delta_phi_' + particle + ' = phi_' + particle + ' - ' + string)
    b = (df_new.loc[df_new['delta_phi_' + particle] > np.pi]).index
    for i in range(0,len(b)):
        df_new.at[b[i],'delta_phi_' + particle] = -6.28318530718 + df_new.at[b[i],'delta_phi_' + particle]
    c = (df_new.loc[df_new['delta_phi_' + particle] < (-1) * np.pi]).index
    for i in range(0,len(c)):
        df_new.at[c[i],'delta_phi_' + particle] = 6.28318530718 + df_new.at[c[i],'delta_phi_' + particle]
    df_new_new = df_new.eval('delta_phi_' + particle + '_abs = abs(delta_phi_' + particle + ')')    
    print 'Fixed ' + particle + ' delta phi range'
    return df_new_new

In [7]:
def getIdAbs(df, var, array):
    for array_element in array:
        a = (df.loc[df[var] == array_element]).index
        for i in range(0,len(a)):
            df.at[a[i], var] = abs(df.at[a[i],var])
        print var + ' of ' + str(array_element) + ' is changed to positive'
    return df

In [3]:
def getDelta(df, particle):
    df.eval('delta_eta_' + particle + ' = eta_' + particle + ' - eta_jet', inplace=True)
    df.eval('delta_pt_' + particle + ' = pt_' + particle + ' - pt_jet', inplace=True)
    df.eval('delta_eta_' + particle + '_abs = abs(delta_eta_' + particle + ')', inplace=True)    
    df.eval('delta_pt_' + particle + '_abs = abs(delta_pt_' + particle + ')', inplace=True)    
    print 'Added colomns for variables of ' + particle + ' jet correlation'
    return df

In [8]:
def toStr(inputArray):
    length = inputArray.size
    array = []
    for i in range(0,length):
        array.append(str(inputArray[i])) 
    return array

In [9]:
def toArrayFloat(begin, end, interval=0, log=False, logNum=0):
    array = []
    if log==True:
        array = np.logspace(begin, end, num=logNum)
    else:
        length = int((end - begin) / interval + 1)
        for i in range(0,length):
            array.append((float(begin + interval * i))) 
    return array

In [10]:
def toArray(begin, end, interval=0, log=False, logNum=0):
    array = []
    if log==True:
        array_float = np.logspace(begin, end, num=logNum)
        array = toStr(array_float)
    else:
        length = int((end - begin) / interval + 1)
        for i in range(0,length):
            array.append(str(float(begin + interval * i))) 
    return array

In [11]:
def bins(df, variable, begin, end, interval=0, log=False, logNum=0):  
    ds=[]
    array = toArray(begin, end, interval, log, logNum)
    for i in range(0,len(array)-1):
        ds.append(df.query('' + str(variable) + ' > ' + str(array[i]) + ' and ' + str(variable) + ' < ' + str(array[i+1]) + ''))
    return ds

In [12]:
def getInterval(arrayFloat):
    interval = np.zeros(len(arrayFloat)-1)
    for i in range(0,len(arrayFloat)-1):
        interval[i] = arrayFloat[i+1] - arrayFloat[i]
    return interval

In [13]:
def normalize(counts, dataframeBeforeCut):
    xsec = np.mean(dataframeBeforeCut['xsec'])
    ntrials = dataframeBeforeCut['ntrials'][dataframeBeforeCut.shape[0]-1]
    return 10 * (counts * xsec * 1e12) / ntrials

In [3]:
def plot1d(df, var, low, high, bottom, top, interval, xlabel, yscale, title, txt):

    xvar_array = toArrayFloat(low, high, interval)
    xsec = np.mean(a['xsec'])
    ntrials = a['ntrials'][a.shape[0]-1]
    
    x, y, xerr = getHist(df, var, xvar_array)
       
    plt.errorbar(x, 10*y*xsec*1e12/ntrials, xerr = xerr, fmt='none', color='b', elinewidth=1)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.yscale(yscale)
    plt.xlabel(xlabel, fontsize=15)
    plt.ylabel('$d\sigma*10fb^{-1}$', fontsize=15)
    plt.title(title, fontsize=15)
    plt.xlim(left=low, right=high)
    plt.ylim(bottom=bottom, top=top)
    plt.legend(prop={'size': 12.5}, frameon=False, loc='best')
    plt.savefig('images/' + txt + '.pdf', bbox='tight')

In [3]:
def plot2d(df, xvar, yvar, xlabel, ylabel, title=None, lim=None):

    plt.hexbin(df[xvar], df[yvar], gridsize=50, norm=LogNorm(), cmap='inferno', xscale = 'log', extent=lim)
    plt.xlabel(xlabel, fontsize = 15)
    plt.ylabel(ylabel, fontsize = 15)
    plt.tick_params(labelsize='large')
    plt.colorbar()
    if title != None:
        plt.title('$' + title + '$', fontsize=15)

In [10]:
def getHist(df, var, binEdges):
        
    x = []
    for i in range(0,len(binEdges)-1):
        x.append((binEdges[i] + binEdges[i+1]) / 2)
    
    y = np.histogram(df[var], bins=binEdges)[0]
    
    xerr = getInterval(binEdges)/2
    
    return x, y, xerr

In [14]:
def rms(array):
    s = 0
    for element in array:
        s = s + element**2
    if len(array) == 0:
        rms = 0
    else:
        rms = np.sqrt(s/len(array))
    return rms

In [7]:
def plot1dnvar(df_array, var_array, label_array, low, high, bottom, top, interval, xlabel, yscale, txt):
    
    color_array = ['r', 'g', 'b', 'purple']
    xvar_array = toArrayFloat(low, high, interval)
    xsec = np.mean(a['xsec'])
    ntrials = a['ntrials'][a.shape[0]-1]
    
    for i in range(0, len(df_array)):
        x, y, xerr = getHist(df_array[i], var_array[i], xvar_array)
        plt.errorbar(x, 10*y*xsec*1e12/ntrials, xerr = xerr, fmt='none', color=color_array[i],elinewidth=2,label=label_array[i])
   
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    plt.yscale(yscale)
    plt.xlabel(xlabel, fontsize=20)
    plt.ylabel('$d\sigma*10fb^{-1}$', fontsize=20)
    #plt.title(title, fontsize=15)
    plt.xlim(left=low, right=high)
    plt.yscale(yscale)
    plt.legend(prop={'size': 20}, frameon=False,loc='upper right')
    plt.savefig('images/' + txt + '.pdf', bbox='tight')

In [4]:
def integrate(binwidth, yarray, low, high):
    s = 0
    interval = (int)((high - low)/binwidth)
    for i in range(0, interval):
        s = s + binwidth * yarray[i]
    return s